In [2]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
import d2lzh_pytorch as d2l
import time
import ipdb
sys.path.append('..')

In [3]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

net = nn.Sequential(
    d2l.FlattenLayer(),
    nn.Linear(num_inputs, num_hiddens),
    nn.ReLU(),
    nn.Linear(num_hiddens, num_outputs)
)

for param in net.parameters():
    print(param.shape, param)
    init.normal_(param, mean=0, std=0.01)
    

torch.Size([256, 784]) Parameter containing:
tensor([[-0.0306,  0.0080,  0.0070,  ..., -0.0326, -0.0030, -0.0108],
        [-0.0004, -0.0029, -0.0078,  ..., -0.0047,  0.0100, -0.0255],
        [-0.0217,  0.0154,  0.0163,  ...,  0.0247,  0.0095,  0.0195],
        ...,
        [-0.0104, -0.0287, -0.0130,  ...,  0.0207,  0.0169,  0.0191],
        [ 0.0202,  0.0307, -0.0254,  ...,  0.0076,  0.0107, -0.0226],
        [ 0.0229,  0.0016,  0.0175,  ...,  0.0077,  0.0025, -0.0151]],
       requires_grad=True)
torch.Size([256]) Parameter containing:
tensor([ 2.3848e-03,  2.2461e-02, -1.0312e-02, -2.2266e-03, -2.7373e-02,
         2.9874e-02,  1.5523e-03,  5.5483e-03,  2.0986e-02, -3.1429e-02,
         3.0219e-02, -3.2412e-02, -1.5186e-05, -1.0035e-02, -3.1499e-02,
        -3.4964e-02, -2.7505e-02, -7.8909e-03,  7.2820e-03, -3.3773e-02,
         7.6040e-03, -1.1011e-02, -2.3746e-02,  1.3016e-03,  2.3907e-02,
         4.6094e-03, -9.6413e-03, -1.1833e-02,  2.4188e-02,  3.1679e-03,
        -2.0344e

In [5]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = .0, 0
    with torch.no_grad():
        for X, y in data_iter:
            net.eval()
            acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            net.train()
            n += y.shape[0]
        net.eval()
    return acc_sum / n

In [ ]:
# train model
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
num_epochs = 5
n = 0

for e in range(num_epochs):
    train_l_loss, train_acc_sum, n = 0, 0, 0
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y).sum()
        # clear grad 
        optimizer.zero_grad()
        # back-prop
        l.backward()
        train_l_loss += l.item()
        train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
        n += y.shape[0]
        ipdb.set_trace()
    print('epoch %d, loss: %.4f train acc: %.4f test acc: %.4f' % (e + 1, train_l_loss / n, train_acc_sum / n, evaluate_accuracy(test_iter, net)))

